# <span style="color:green"><center>Diplomado en Inteligencia Artificial y Aprendizaje Profundo</center></span>

# <span style="color:red"><center>Canalización  de datos. La API tf.data</center></span>

##   <span style="color:blue">Profesores</span>

1. Alvaro Mauricio Montenegro Díaz, ammontenegrod@unal.edu.co
2. Daniel Mauricio Montenegro Reyes, dextronomo@gmail.com 
3. Campo Elías Pardo Turriago, cepardot@unal.edu.co 

##   <span style="color:blue">Asesora Medios y Marketing digital</span>
 

4. Maria del Pilar Montenegro, pmontenegro88@gmail.com 

## <span style="color:blue">Asistentes</span>

5. Oleg Jarma, ojarmam@unal.edu.co 
6. Laura Lizarazo, ljlizarazore@unal.edu.co 

## <span style="color:blue">Contenido</span> 

## <span style="color:blue">Introducción</span> 

Basado en [tf.data](https://www.tensorflow.org/guide/data).

La API `tf.data` permite crear tuberías de entrada complejas a partir de piezas simples y reutilizables. Por ejemplo, la canalización de un modelo de imagen podría agregar datos de archivos en un sistema de archivos distribuido, aplicar perturbaciones aleatorias a cada imagen y fusionar imágenes seleccionadas al azar en un lote para entrenamiento. La canalización de un modelo de texto puede implicar extraer símbolos de datos de texto sin procesar, convertirlos en identificadores incrustados con una tabla de búsqueda y agrupar secuencias de diferentes longitudes. 

## <span style="color:blue">Importa librerías</span> 

In [1]:
import tensorflow as tf

import pathlib
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

np.set_printoptions(precision=4)

## <span style="color:blue">Esenciales</span> 

Para crear una canalización de entrada, debe comenzar con una fuente de datos. Por ejemplo, para construir un `Dataset` de datos a partir de datos en la memoria, puede usar *tf.data.Dataset.from_tensors()* o *tf.data.Dataset.from_tensor_slices()*. Alternativamente, si sus datos de entrada están almacenados en un archivo en el formato *TFRecord* de TensorFlow puede usar *tf.data.TFRecordDataset()*.

El objeto Dataset es un iterable de Python. Esto hace posible consumir sus elementos usando un bucle for:

In [2]:
dataset = tf.data.Dataset.from_tensor_slices([8, 3, 0, 8, 2, 1])
dataset

<TensorSliceDataset shapes: (), types: tf.int32>

In [3]:
len(dataset)

6

In [4]:
for elem in dataset:
    print(elem.numpy())

8
3
0
8
2
1


o se pueden crear explícitamente un iterador

In [5]:
it = iter(dataset)
print(next(it).numpy())
print(next(it).numpy())

8
3


### Consumo de datos usando reducción: reduce

In [6]:
print(dataset.reduce(0, lambda state, value: state+value).numpy())

22


## <span style="color:blue">Estructura del conjunto de datos</span> 

Un conjunto de datos produce una secuencia de elementos , donde cada elemento tiene la misma estructura (anidada) de componentes . 

Los componentes individuales de la estructura pueden ser de cualquier tipo representable por *tf.TypeSpec*, incluidos *tf.Tensor* , *tf.sparse.SparseTensor* ,*tf.RaggedTensor* , *tf.TensorArray* o *tf.data.Dataset*.

Las construcciones de Python que se pueden usar para expresar la estructura (anidada) de elementos incluyen *tuple , dict , NamedTuple y OrderedDict*. 

En particular, *list* no es una construcción válida para expresar la estructura de los elementos del conjunto de datos. 

Si desea que una entrada de *list* se trate como una estructura, debe convertirla en tuple y si desea que una lista de salida, entonces debe empaquetarla explícitamente usando *tf.stack*.


La propiedad *Dataset.element_spec* permite inspeccionar el tipo de cada componente del elemento. La propiedad devuelve una estructura anidada de objetos *tf.TypeSpec*, que coincide con la estructura del elemento, que puede ser un solo componente, una tupla de componentes o una tupla anidada de componentes. Por ejemplo:

In [7]:
dataset1 = tf.data.Dataset.from_tensor_slices(tf.random.uniform([4, 10]))
print(dataset1)

<TensorSliceDataset shapes: (10,), types: tf.float32>


In [8]:
for i in dataset1:
    print(i.numpy())

[0.7701 0.552  0.7585 0.4115 0.8178 0.644  0.5679 0.1752 0.4314 0.8794]
[0.2899 0.756  0.6285 0.7384 0.1255 0.4635 0.6919 0.9342 0.3346 0.502 ]
[0.3587 0.9728 0.4452 0.4349 0.2622 0.8186 0.6079 0.0305 0.0648 0.0877]
[0.8672 0.6902 0.4152 0.436  0.0207 0.2833 0.8699 0.324  0.7565 0.297 ]


In [9]:
len(dataset1)

4

In [10]:
dataset2 = tf.data.Dataset.from_tensor_slices(
    (tf.random.uniform([4]),
     tf.random.uniform([4,100], maxval=100, dtype=tf.int32)))

dataset2.element_spec
    

(TensorSpec(shape=(), dtype=tf.float32, name=None),
 TensorSpec(shape=(100,), dtype=tf.int32, name=None))

In [11]:
len(dataset2)

4

In [11]:
dataset3 = tf.data.Dataset.zip((dataset1, dataset2))
dataset3.element_spec

(TensorSpec(shape=(10,), dtype=tf.float32, name=None),
 (TensorSpec(shape=(), dtype=tf.float32, name=None),
  TensorSpec(shape=(100,), dtype=tf.int32, name=None)))

In [12]:
len(dataset3)

4

In [13]:
type(dataset3)

tensorflow.python.data.ops.dataset_ops.ZipDataset

In [14]:
i = iter(dataset3)

In [15]:
print(i.next(), "\n")
print(i.next(), "\n")
print(i.next(), "\n")
print(i.next(), "\n")

(<tf.Tensor: shape=(10,), dtype=float32, numpy=
array([0.005 , 0.8077, 0.5108, 0.9167, 0.8164, 0.3464, 0.8314, 0.5843,
       0.3102, 0.061 ], dtype=float32)>, (<tf.Tensor: shape=(), dtype=float32, numpy=0.42063427>, <tf.Tensor: shape=(100,), dtype=int32, numpy=
array([94, 69, 73, 69, 12, 22, 78, 91, 27, 35, 72,  2, 47, 65, 49, 48, 31,
       60, 81, 18, 47,  9, 25,  0, 51,  4, 46, 66, 47, 67, 91, 21, 41, 11,
       25, 92, 79, 57, 43, 84, 14, 42, 36, 27,  4, 82, 39, 98, 34, 60, 44,
       30, 83, 55, 82, 48,  9, 53, 43, 44, 20, 37, 78, 81, 80, 39, 27, 72,
       76, 86, 51,  5, 18, 22, 51, 19, 14, 22,  1, 84, 48, 83, 48, 68, 25,
       32, 15, 44, 62, 67, 10, 68, 86, 76,  2, 76, 98,  8, 52, 97],
      dtype=int32)>)) 

(<tf.Tensor: shape=(10,), dtype=float32, numpy=
array([0.8523, 0.9912, 0.8094, 0.8738, 0.2482, 0.9336, 0.6535, 0.4829,
       0.6793, 0.1769], dtype=float32)>, (<tf.Tensor: shape=(), dtype=float32, numpy=0.47015595>, <tf.Tensor: shape=(100,), dtype=int32, numpy=
array([

In [16]:
# dataset con tensores dispersos
dataset4 = tf.data.Dataset.from_tensors(tf.SparseTensor(indices=[[0, 0],[1, 2]], values=[1, 2], dense_shape=[3, 4]))
dataset4.element_spec

SparseTensorSpec(TensorShape([3, 4]), tf.int32)

In [17]:
dataset4.element_spec.value_type

tensorflow.python.framework.sparse_tensor.SparseTensor

In [58]:
for a, (b,c) in dataset3:
    print('shapes: {a.shape}, {b.shape}, {c.shape}'.format(a=a, b=b, c=c))

shapes: (10,), (), (100,)
shapes: (10,), (), (100,)
shapes: (10,), (), (100,)
shapes: (10,), (), (100,)


## <span style="color:blue">Leer datos de entrada</span> 

### Consumir matrices Numpy

Si todos sus datos de entrada caben en la memoria, la forma más sencilla de crear un Dataset a partir de ellos es convertirlos en objetos tf.Tensor y usar Dataset.from_tensor_slices() .

In [18]:
train, test = tf.keras.datasets.fashion_mnist.load_data()

In [19]:
imagenes, labels  = train
imagenes = imagenes /255.

dataset = tf.data.Dataset.from_tensor_slices((imagenes, labels))
dataset

<TensorSliceDataset shapes: ((28, 28), ()), types: (tf.float64, tf.uint8)>

### Consumir generadores de Python

In [4]:
def count(stop):
    i=0
    while i<stop:
        yield i
        i+= 1
        
for n in count(5):
    print(n)

0
1
2
3
4


El constructor `Dataset.from_generator` convierte el generador de Python en un `tf.data.Dataset` completamente funcional.

El constructor toma un invocable como entrada, no un iterador. Esto le permite reiniciar el generador cuando llega al final. Toma un argumento args opcional, que se pasa como argumentos del invocable.

El argumento *output_types* es necesario porque *tf.data* crea un *tf.Graph* internamente y los bordes del gráfico requieren un tf.dtype .




In [5]:
ds_counter = tf.data.Dataset.from_generator(count, args=[25], output_types=tf.int32, output_shapes=(),)

In [65]:
for count_batch in ds_counter.repeat().batch(10).take(10):
    print(count_batch.numpy())

[0 1 2 3 4 5 6 7 8 9]
[10 11 12 13 14 15 16 17 18 19]
[20 21 22 23 24  0  1  2  3  4]
[ 5  6  7  8  9 10 11 12 13 14]
[15 16 17 18 19 20 21 22 23 24]
[0 1 2 3 4 5 6 7 8 9]
[10 11 12 13 14 15 16 17 18 19]
[20 21 22 23 24  0  1  2  3  4]
[ 5  6  7  8  9 10 11 12 13 14]
[15 16 17 18 19 20 21 22 23 24]


El argumento `output_shapes` no es necesario, pero se recomienda, ya que muchas operaciones de flujo tensorial no admiten tensores con rango desconocido. Si la longitud de un eje en particular es desconocida o variable, output_shapes puede colcarse como None.

También es importante tener en cuenta que `output_shapes` y `output_types` siguen las mismas reglas de anidamiento que otros métodos de conjuntos de datos.

Aquí hay un generador de ejemplo que demuestra ambos aspectos, devuelve tuplas de matrices, donde la segunda matriz es un vector con longitud desconocida.

In [6]:
def gen_series():
    i = 0
    while True:
        size = np.random.randint(0,10)
        yield i, np.random.normal(size = (size,))
        i+=1

In [7]:
for i, series in gen_series():
    print(i, ":", str(series))
    if i > 5:
        break

0 : [-0.589  -0.1256  0.1138  0.6881 -1.1627  1.226   0.3724 -0.1009]
1 : [ 0.4862  2.7924 -0.5485  0.4517 -0.8274 -0.5066 -0.079 ]
2 : [-2.0796e+00 -1.7316e-05  1.2649e+00  1.2178e+00 -6.4795e-02 -1.2795e+00
 -2.1617e+00 -5.6905e-02]
3 : [-0.2579  1.3068  0.4814]
4 : [0.8903 1.1579]
5 : [ 1.1675 -0.9602  0.8503  0.0468]
6 : [ 1.1075  0.1869  1.099  -1.0206 -0.0775]


La primera salida es un *int32* la segunda es un *float32*.

El primer elemento es un escalar, forma () , y el segundo es un vector de longitud desconocida, forma (None,)


In [8]:
ds_series = tf.data.Dataset.from_generator(
    gen_series,
    output_types=(tf.int32, tf.float32),
    output_shapes=((), (None, )))

ds_series

<FlatMapDataset shapes: ((), (None,)), types: (tf.int32, tf.float32)>

Ahora se puede utilizar como un *tf.data.Dataset* normal. Tenga en cuenta que al procesar por lotes un conjunto de datos con una forma variable, debe usar *Dataset.padded_batch*.

In [9]:
ds_series_batch = ds_series.shuffle(20).padded_batch(10)

ids, sequence_batch = next(iter(ds_series_batch))

print (ids.numpy())
print()
print(sequence_batch.numpy())

[ 7 11 10 12  6 17 24  2 27  9]

[[-1.4308e+00  1.2503e-03  1.3745e+00 -2.3217e+00  7.8626e-02  3.0368e-01
  -8.8505e-01  6.8241e-01]
 [ 2.7771e-01 -2.8299e-01  5.7397e-01  0.0000e+00  0.0000e+00  0.0000e+00
   0.0000e+00  0.0000e+00]
 [ 0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00
   0.0000e+00  0.0000e+00]
 [ 4.7146e-01  1.0614e-02  6.7689e-01  0.0000e+00  0.0000e+00  0.0000e+00
   0.0000e+00  0.0000e+00]
 [ 1.5210e+00  3.3249e-01  2.0336e-01 -1.2920e+00  0.0000e+00  0.0000e+00
   0.0000e+00  0.0000e+00]
 [-4.5420e-01 -5.6538e-02  2.1903e+00 -8.4784e-02  3.2478e-01 -1.7719e+00
  -1.0545e+00  5.3183e-01]
 [-3.9575e-01 -7.3953e-01 -1.9150e+00  1.9243e+00  6.4415e-01  1.3259e+00
   6.1047e-01  7.0616e-01]
 [-5.2022e-01 -3.6112e-01 -5.0845e-01 -5.7890e-01  9.5692e-01  0.0000e+00
   0.0000e+00  0.0000e+00]
 [ 6.5591e-01  2.2062e+00  8.2903e-01  4.3211e-01 -6.5034e-01 -2.1691e-01
   0.0000e+00  0.0000e+00]
 [-5.7040e-01 -9.4705e-01  1.2774e+00  0.0000e+00  0.0000e

In [10]:
it = iter(ds_series_batch)
for i in range(10):
    ids, sequence_batch = next(it)
    print (ids.numpy())
    print()
    print(sequence_batch.numpy())
    print()
    

[ 8 15  3  0 18  1 14 16 27  5]

[[-1.061  -0.9733 -1.086  -0.9015 -0.0504  0.      0.      0.      0.    ]
 [-0.0574  0.7203 -0.9158  1.0477  0.      0.      0.      0.      0.    ]
 [ 0.      0.      0.      0.      0.      0.      0.      0.      0.    ]
 [-0.622   1.1692 -1.946   0.0864 -0.8531  0.711  -1.5698  3.2136  0.5569]
 [-0.65    0.4697 -0.6727 -0.9256 -2.3336 -0.872  -0.1654  0.      0.    ]
 [-0.5982 -0.3164  0.      0.      0.      0.      0.      0.      0.    ]
 [ 1.4281  0.      0.      0.      0.      0.      0.      0.      0.    ]
 [ 0.4866 -0.5384 -0.3614 -0.1805 -1.3637 -1.2998  0.      0.      0.    ]
 [-1.4777 -0.8462  0.6291  0.8153 -0.7571  0.7379 -0.3234  0.      0.    ]
 [-1.0155 -0.2286  0.4975 -1.0118  0.9367 -0.2049  0.9626 -1.4553  0.    ]]
[20 28  4  7 10 26 19 33 34 37]

[[ 0.6415  0.2909  0.6327  0.7194  0.9924  0.3216  0.9457  1.3295 -0.297 ]
 [ 0.      0.      0.      0.      0.      0.      0.      0.      0.    ]
 [ 1.4123  0.249  -0.3707 -1.1176

### Ejemplo realista con imágenes

Para obtener un ejemplo más realista, intente `tf.data.Dataset` `preprocessing.image.ImageDataGenerator` como un `tf.data.Dataset` .

Primero descargue los datos:


In [11]:
flowers = tf.keras.utils.get_file(
    'flower_photos',
    'https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz',
    untar=True)

228818944/228813984 [==============================] - 100s 0us/step


In [18]:
print(flowers)

/home/alvaro/.keras/datasets/flower_photos


Cree la `image.ImageDataGenerator`

In [12]:
image_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255, rotation_range=20)

In [13]:
images, labels = next(image_gen.flow_from_directory(flowers))

Found 3670 images belonging to 5 classes.


In [14]:
print(images.dtype, images.shape)
print(labels.dtype, labels.shape)

float32 (32, 256, 256, 3)
float32 (32, 5)


In [16]:
ds = tf.data.Dataset.from_generator(
    lambda: image_gen.flow_from_directory(flowers),
    output_types=(tf.float32, tf.float32),
    output_shapes=([32,256,256,3],[32,5]))

ds.element_spec

(TensorSpec(shape=(32, 256, 256, 3), dtype=tf.float32, name=None),
 TensorSpec(shape=(32, 5), dtype=tf.float32, name=None))

### Consumir datos de texto

Muchos conjuntos de datos se distribuyen como uno o más archivos de texto. `tf.data.TextLineDataset` proporciona una manera fácil de extraer líneas de uno o más archivos de texto. 

Dados uno o más nombres de archivo, un `TextLineDataset` producirá un elemento con valor de cadena por línea de esos archivos.

In [20]:
directory_url = 'https://storage.googleapis.com/download.tensorflow.org/data/illiad/'
file_names = ['cowper.txt', 'derby.txt', 'butler.txt']

file_paths = [
    tf.keras.utils.get_file(file_name, directory_url +file_name)
    for file_name in file_names
]

811008/807992 [==============================] - 1s 1us/step


In [24]:
file_paths

['/home/alvaro/.keras/datasets/cowper.txt',
 '/home/alvaro/.keras/datasets/derby.txt',
 '/home/alvaro/.keras/datasets/butler.txt']

In [21]:
dataset = tf.data.TextLineDataset(file_paths)

Estas son las primeras líneas del primer archivo:

In [22]:
for line in dataset.take(5):
    print(line.numpy())

b"\xef\xbb\xbfAchilles sing, O Goddess! Peleus' son;"
b'His wrath pernicious, who ten thousand woes'
b"Caused to Achaia's host, sent many a soul"
b'Illustrious into Ades premature,'
b'And Heroes gave (so stood the will of Jove)'


Para alternar líneas entre archivos, use `Dataset.interleave` . Esto facilita la reproducción aleatoria de archivos. Aquí están la primera, segunda y tercera líneas de cada traducción:

In [26]:
file_ds = tf.data.Dataset.from_tensor_slices(file_paths)

In [29]:
for i in file_ds: print(i.numpy())

b'/home/alvaro/.keras/datasets/cowper.txt'
b'/home/alvaro/.keras/datasets/derby.txt'
b'/home/alvaro/.keras/datasets/butler.txt'


In [ ]:
line_ds = files_ds.interleave(tf.data.TextLineDataset, cycle_length=3)

for i, line in enumerate(lines_ds.take(9)):
    if i%3 ==0:


In [55]:
t = tf.constant([
  [[0, 1, 2, 3, 4],
   [5, 6, 7, 8, 9]],
  [[10, 11, 12, 13, 14],
   [15, 16, 17, 18, 19]],
  [[20, 21, 22, 23, 24],
   [25, 26, 27, 28, 29]],]) 
print(t.numpy())

[[[ 0  1  2  3  4]
  [ 5  6  7  8  9]]

 [[10 11 12 13 14]
  [15 16 17 18 19]]

 [[20 21 22 23 24]
  [25 26 27 28 29]]]


In [56]:
print('Extrae la primera capa\n', t[0,:,:])
print('Extrae la segunda capa\n', t[1])

Extrae la primera capa
 tf.Tensor(
[[0 1 2 3 4]
 [5 6 7 8 9]], shape=(2, 5), dtype=int32)
Extrae la segunda capa
 tf.Tensor(
[[10 11 12 13 14]
 [15 16 17 18 19]], shape=(2, 5), dtype=int32)


## <span style="color:blue">Manipular formas</span>

In [34]:
x = tf.constant([[1],[2],[3]])
print(x.shape)
print(x.shape.as_list())

(3, 1)
[3, 1]


In [35]:
reshaped = tf.reshape(x,[1,3])
print(reshaped.numpy())
print(x.numpy())

[[1 2 3]]
[[1]
 [2]
 [3]]


Con *tf.reshape* los datos mantienen su disposición en la memoria y se crea un nuevo tensor, con la forma solicitada, apuntando a los mismos datos. TensorFlow usa un orden de memoria de "fila principal" de estilo C, donde incrementar el índice de la derecha corresponde a un solo paso en la memoria.

![tensor before](../Imagenes/tensor_before.png)

Tensor antes de reshape

<img src="../Imagenes/tensor_after.png" align="left" width="40%">
<img src="../Imagenes/tensor_after2.png" align="right" width="30%">

Tensores despues de reshape

### Aplana un tensor

Esta operación permite ver el orden como están organizados los datos en memoria

In [38]:
print("t actual: \n", t.numpy())
flat = tf.reshape(t, [-1])
print("\n t aplanado: \n", flat.numpy())

t actual: 
 [[[ 1  2  3  4  5]
  [ 6  7  8  9 10]
  [11 12 13 14 15]]

 [[16 17 18 19 20]
  [21 22 23 24 25]
  [26 27 28 29 30]]]

 t aplanado: 
 [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30]


Normalmente, el único uso razonable de tf.reshape es combinar o dividir ejes adyacentes (o agregar / eliminar 1 s).

Para este tensor de 2x3x5, remodelar a (2x3)x5 o 2x (3x5) son dos cosas razonables, ya que los cortes no se mezclan:


In [39]:
print(t.shape.as_list())

[2, 3, 5]


In [42]:
print(tf.reshape(t, [2*3,5]))

tf.Tensor(
[[ 1  2  3  4  5]
 [ 6  7  8  9 10]
 [11 12 13 14 15]
 [16 17 18 19 20]
 [21 22 23 24 25]
 [26 27 28 29 30]], shape=(6, 5), dtype=int32)


In [43]:
print(tf.reshape(t, [2,3*5]))

tf.Tensor(
[[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
 [16 17 18 19 20 21 22 23 24 25 26 27 28 29 30]], shape=(2, 15), dtype=int32)


No es necesario definir todos tamaños en todas las dimensiones. veámos como rehacer los dos ejemplos ateriores, respecitivamente. El -1 le dice a tf que decida cuál es la dimensión correcta.

In [57]:
print(tf.reshape(t, [-1,5]))

tf.Tensor(
[[ 0  1  2  3  4]
 [ 5  6  7  8  9]
 [10 11 12 13 14]
 [15 16 17 18 19]
 [20 21 22 23 24]
 [25 26 27 28 29]], shape=(6, 5), dtype=int32)


In [58]:
print(tf.reshape(t, [3,-1]))

tf.Tensor(
[[ 0  1  2  3  4  5  6  7  8  9]
 [10 11 12 13 14 15 16 17 18 19]
 [20 21 22 23 24 25 26 27 28 29]], shape=(3, 10), dtype=int32)


Aunque es posible hacer el siguiente reshape no tiene sentido, porque se pierde la integridad de la información. 

Asegúrese que enti3nde la razón.

In [47]:
print(tf.reshape(t, [3,2,5]))

tf.Tensor(
[[[ 1  2  3  4  5]
  [ 6  7  8  9 10]]

 [[11 12 13 14 15]
  [16 17 18 19 20]]

 [[21 22 23 24 25]
  [26 27 28 29 30]]], shape=(3, 2, 5), dtype=int32)


### Definición Conversión de tipos. Cast

In [48]:
f64_tensor = tf.constant([2.0, 4.0, 6.0], dtype = tf.float64)
print(f64_tensor)

f16_tensor = tf.cast(f64_tensor,dtype= tf.float16)
print(f16_tensor)

u8_tensor = tf.cast(f16_tensor, dtype = tf.uint8)
print(u8_tensor)

tf.Tensor([2. 4. 6.], shape=(3,), dtype=float64)
tf.Tensor([2. 4. 6.], shape=(3,), dtype=float16)
tf.Tensor([2 4 6], shape=(3,), dtype=uint8)


## <span style="color:blue">Radiofusión (broadcasting)</span>

La radiodifusión es un concepto tomado de la función equivalente en NumPy . En resumen, bajo ciertas condiciones, los tensores más pequeños se "estiran" automáticamente para adaptarse a tensores más grandes cuando se ejecutan operaciones combinadas en ellos.

El caso más simple y común es cuando intenta multiplicar o agregar un tensor a un escalar. En ese caso, el escalar se transmite para que tenga la misma forma que el otro argumento.

In [49]:
x = tf.constant([1, 2 ,3])
y = tf.constant(2)
z = tf.constant([2, 2 ,2])

# el mismo resultado
print(tf.multiply(x,2))
print(x*y)
print(x*z)

tf.Tensor([2 4 6], shape=(3,), dtype=int32)
tf.Tensor([2 4 6], shape=(3,), dtype=int32)
tf.Tensor([2 4 6], shape=(3,), dtype=int32)


In [50]:
x = tf.reshape(x, [3,1])
y = tf.range(1,5)
print(x, "\n")
print(y, "\n")

tf.Tensor(
[[1]
 [2]
 [3]], shape=(3, 1), dtype=int32) 

tf.Tensor([1 2 3 4], shape=(4,), dtype=int32) 



In [52]:
print(tf.multiply(x,y))

tf.Tensor(
[[ 1  2  3  4]
 [ 2  4  6  8]
 [ 3  6  9 12]], shape=(3, 4), dtype=int32)


Del mismo modo, los ejes con longitud 1 se pueden estirar para que coincidan con los otros argumentos. Ambos argumentos se pueden estirar en el mismo cálculo.

En este caso, una matriz de 3x1 se multiplica por elementos por una matriz de 1x4 para producir una matriz de 3x4. Observe que el 1 inicial es opcional: la forma de y es [4]. En matemáticas esta multiplicación se conoce como producto externo.

### tf.broadcast_to

In [53]:
print(tf.broadcast_to([1,2,3], [3,3]))

tf.Tensor(
[[1 2 3]
 [1 2 3]
 [1 2 3]], shape=(3, 3), dtype=int32)


### tf.convert_to_tensor

La mayoría de las operaciones, como *tf.matmul* y *tf.reshape* toman argumentos de la clase *tf.Tensor*. Sin embargo, notará que en el caso anterior, se aceptan objetos de Python con forma de tensores.

La mayoría, pero no todas, las operaciones llaman a *tf.convert_to_tensor* con argumentos no tensoriales. Existe un registro de conversiones, y la mayoría de las clases de objetos como *ndarray* , *TensorShape* , de Python, y *tf.Variable* se convertirán todas automáticamente.



## <span style="color:blue">Tensores irregulares (ragged tensors) </span>

Un tensor con números variables de elementos a lo largo de algún eje se llama "irregular". Utilice tf.ragged.RaggedTensor para datos irregulares.

Por ejemplo, esto no se puede representar como un tensor regular:



![ragged](../Imagenes/ragged.png)

In [59]:
ragged_list = [
    [0, 1, 2, 3],
    [4, 5],
    [6, 7, 8],
    [9]]

In [64]:
try:
    tensor = tf.constant(ragged_list)
except Exception as e:
     print(f"{type(e).__name__}: {e}")



ValueError: Can't convert non-rectangular Python sequence to Tensor.


En su lugar, cree un *tf.RaggedTensor* usando *tf.ragged.constant* :

In [65]:
ragged_t = tf.ragged.constant(ragged_list)

print(ragged_t.shape)

(4, None)


## <span style="color:blue">Tensores de strings</span>

In [66]:
st = tf.constant("Este tensor string")
print(st)

tf.Tensor(b'Este tensor string', shape=(), dtype=string)


In [69]:
st = tf.constant(["Este tensor string",
                 "Cadena 2",
                 "Cadena 3",
                 "🥳"])
print(st)

tf.Tensor([b'Este tensor string' b'Cadena 2' b'Cadena 3' b'\xf0\x9f\xa5\xb3'], shape=(4,), dtype=string)


In [ ]:
El prefijo b indica que el tipo (dtype)  tf.string no es unicode.

In [70]:
print(tf.strings.split(st))

<tf.RaggedTensor [[b'Este', b'tensor', b'string'], [b'Cadena', b'2'], [b'Cadena', b'3'], [b'\xf0\x9f\xa5\xb3']]>


In [73]:
st_split = tf.strings.split(st)
for i in st_split:
    print(i)

tf.Tensor([b'Este' b'tensor' b'string'], shape=(3,), dtype=string)
tf.Tensor([b'Cadena' b'2'], shape=(2,), dtype=string)
tf.Tensor([b'Cadena' b'3'], shape=(2,), dtype=string)
tf.Tensor([b'\xf0\x9f\xa5\xb3'], shape=(1,), dtype=string)


In [ ]:
### string to number

In [74]:
st = tf.constant("1 10 10.4")

print(tf.strings.to_number(tf.strings.split(st, " ")))

tf.Tensor([ 1.  10.  10.4], shape=(3,), dtype=float32)


## <span style="color:blue">Tensores dispersos.  SparseTensor</span>

In [75]:
#  tensor disperso
sparse_tensor = tf.sparse.SparseTensor(indices = [[0,1], [1,2]],
                                       values = [1,2],
                                       dense_shape
                                        =[3,4])
print(sparse_tensor, "\n")

# convierte a tensor denso
print(tf.sparse.to_dense(sparse_tensor))

SparseTensor(indices=tf.Tensor(
[[0 1]
 [1 2]], shape=(2, 2), dtype=int64), values=tf.Tensor([1 2], shape=(2,), dtype=int32), dense_shape=tf.Tensor([3 4], shape=(2,), dtype=int64)) 

tf.Tensor(
[[0 1 0 0]
 [0 0 2 0]
 [0 0 0 0]], shape=(3, 4), dtype=int32)


## <span style="color:blue">zip</span>

In [43]:
a = tf.data.Dataset.range(1,4) # ==> [1, 2, 3]
b = tf.data.Dataset.range(4,7) # ==> [4, 5, 6]

ds = tf.data.Dataset.zip((a, b))

In [41]:
for i in a:
    print(i.numpy())

1
2
3


In [46]:
for i in ds:
    print(i)

(<tf.Tensor: shape=(), dtype=int64, numpy=1>, <tf.Tensor: shape=(), dtype=int64, numpy=4>)
(<tf.Tensor: shape=(), dtype=int64, numpy=2>, <tf.Tensor: shape=(), dtype=int64, numpy=5>)
(<tf.Tensor: shape=(), dtype=int64, numpy=3>, <tf.Tensor: shape=(), dtype=int64, numpy=6>)


In [48]:
list(ds.as_numpy_iterator())

[(1, 4), (2, 5), (3, 6)]

In [49]:
c = tf.data.Dataset.range(7,11) # ==> [7, 8, 9, 10]

In [53]:
ds1 = tf.data.Dataset.zip((c,b,a))
list(ds1.as_numpy_iterator())

[(7, 4, 1), (8, 5, 2), (9, 6, 3)]